In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from torch.optim import *
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from flaml import AutoML
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,precision_score,f1_score,recall_score
import pickle
import wandb
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
PROJECT_NAME = 'Dogecoin-Price-Prediction-Sklearn'
device = 'cuda'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-08-24,0.001731,0.001845,0.001722,0.001816,0.001816,4796720.0
1,2017-08-25,0.001808,0.001830,0.001762,0.001790,0.001790,2654040.0
2,2017-08-26,0.001787,0.001797,0.001735,0.001778,0.001778,1935350.0
3,2017-08-27,0.001779,0.001784,0.001745,0.001760,0.001760,2241620.0
4,2017-08-28,0.001764,0.001825,0.001751,0.001825,0.001825,3210440.0


In [4]:
len(data)

1462

In [5]:
data.dropna(inplace=True)

In [6]:
X = data['Date']
y = data['Open']

In [7]:
new_X = []

In [8]:
X[0].split('-')

['2017', '08', '24']

In [9]:
for x in X:
    x = x.split('-')
    x = f'{x[0]}{x[1]}{x[2]}'
    x = int(x)
    new_X.append(x)

In [10]:
data['Date'] = new_X

In [11]:
X = data['Date']
y = data['Open']

In [12]:
def valid(model,X,y,valid=False):
    preds = model.predict(X)
    try:
        if valid:
            result = {
                'Val MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'Val MSE':mean_squared_error(y_pred=preds,y_true=y),
                'Val Accuracy':model.score(X,y)
            }
        else:
            result = {
                'MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'MSE':mean_squared_error(y_pred=preds,y_true=y),
                'Accuracy':model.score(X,y)
            }
    except:
        if valid:
            result = {
                'Val MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'Val MSE':mean_squared_error(y_pred=preds,y_true=y),
            }
        else:
            result = {
                'MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'MSE':mean_squared_error(y_pred=preds,y_true=y),
            }
    return result

In [13]:
def fit(model,X,y,name):
    X = np.array(X)
    y = np.array(y)
    wandb.init(project=PROJECT_NAME,name=name)
    model.fit(X.reshape(-1, 1),y.reshape(-1, 1))
    wandb.log(valid(model,X.reshape(-1, 1),y.reshape(-1, 1)))
    pickle.dump(model,open(f'/home/indika/Programming/Projects/Python/Artifical-Intelligence/Sklearn/Regression/Dogecoin-Price-Prediction/{name}.pkl','wb'))
    return model

In [14]:
fit(GradientBoostingRegressor(),X,y,'baseline')

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor()

In [15]:
preprocessings = [StandardScaler,RobustScaler,MinMaxScaler,MaxAbsScaler,Normalizer]

In [16]:
X_old = pd.DataFrame(X).copy()

In [18]:
# for preproccessing in preprocessings:
#     X = X_old.copy()
#     ct = make_column_transformer(
#         (preproccessing,['Date'])
#     )
#     X = ct.fit_transform(X)
#     fit(GradientBoostingRegressor(),X,y,f'preproccessing-{preproccessing}')

In [19]:
models = [
    ['KNeighborsRegressor',KNeighborsRegressor],
    ['LogisticRegression',LogisticRegression],
    ['LogisticRegressionCV',LogisticRegressionCV],
    ['DecisionTreeRegressor',DecisionTreeRegressor],
    ['GradientBoostingRegressor',GradientBoostingRegressor],
    ['AdaBoostRegressor',AdaBoostRegressor],
    ['RandomForestRegressor',RandomForestRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['GaussianNB',GaussianNB],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
    ['CatBoost',CatBoost],
    ['CatBoostRegressor',CatBoostRegressor],
    ['XGBRegressor',XGBRegressor],
    ['XGBRFRegressor',XGBRFRegressor],
    ['ExtraTreesRegressor',ExtraTreesRegressor],
]

In [20]:
for model in models:
    fit(model[1](),X,y,f'model-{model[0]}')

MAE,0.00199
MSE,7e-05
Accuracy,0.99195
_runtime,11
_timestamp,1629816788
_step,0


MAE,▁
MSE,▁
Accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


MAE,0.00177
MSE,7e-05
Accuracy,0.99219
_runtime,6
_timestamp,1629816952
_step,0


MAE,▁
MSE,▁
Accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: Unknown label type: 'continuous'